In [1]:
import os
os.chdir("/Users/karol/Desktop/Antwerp/ai_project")
import torch
import torch
import torch.nn as nn
from model import MLP1, TwoTower
from data_reader import load_data, data_preprocessing, load_data_mf, customer_buckets
from helper import validate_softmax,  train_softmax, train_two_tower
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Data Preprocessing

### Preprocess and save data

In [ ]:
# transactions, articles, customers, article_encodings, customer_encodings = data_preprocessing(feature_generation=False, return_encodings=True, save=True)

### Load preprocessed data and apply one hot encoding for articles and customers

In [2]:
# read data
transactions = pd.read_csv("data/preprocessed/transactions.csv") 
articles = pd.read_csv("data/preprocessed/articles.csv") 
customers = pd.read_csv("data/preprocessed/customers.csv") 

# one hot encoding 
articles = articles.set_index("article_id")
customers = customers.set_index("customer_id")

article_enc = OneHotEncoder(sparse_output=True)
articles = article_enc.fit_transform(articles)

customers_categorical = ["FN",'Active',"club_member_status", "fashion_news_frequency"]
customers_cont = ["age"]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=True), customers_categorical),
        ('cont', 'passthrough', customers_cont)  # 'passthrough' means no transformation for continuous variables
    ],
    remainder='drop'  # Drop any columns not explicitly transformed
)
customers = csr_matrix(preprocessor.fit_transform(customers))

# Softmax DNN

### Load Data & Create Model

In [ ]:
train_dataloader, val_dataloader = load_data(transactions, train_test=True, batch_size=1000)

### Create & Train basic Model 

In [ ]:
# # create model
# model = MLP1(input_dim=n_articles, output_dim=n_articles)
# # get params
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# save_dir = "models/MLP1.pt"
# # train
# val_loss_list, val_acc_list = train_softmax(model, train_dataloader, val_dataloader, criterion, optimizer, save_dir, num_epochs=5)

### Load trained model

In [ ]:
MLP1 = torch.load("KarolZiolo/RQ1/models/MLP1.pt")
criterion = nn.MSELoss()
#final validation accuracy
val_loss, val_accuracy = validate_softmax(MLP1, val_dataloader, criterion, k=3)
val_accuracy.item()

### Ideas for improvements:
- No warm start
- Model should be trained on customers who had at least two purchases
- We take into account baskets which are based on the past 2 years. Maybe we should train model based on customers who bought articles in last month.
- Different way to measure accuracy.
- Customers whou didn't developed any patterns (bought low amount of clothes) should be recommended with current top selling articles.
- Develop more complex (deeper) model.
- More training. 



# Matrix Factorization with DNN

In [3]:
train_dataloader, val_dataloader = load_data_mf(transactions, batch_size=100)

In [4]:
buckets = customer_buckets(transactions, train_test=False)

In [6]:
input_article_dim = articles.shape[1]
input_customer_dim = customers.shape[1]
model = TwoTower(input_article_dim, input_customer_dim, output_dim=3)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
save_dir = "KarolZiolo/RQ1/models/TwoTower1.pt"
train_two_tower(model, customers, articles, buckets, train_dataloader, val_dataloader, criterion, optimizer, save_dir, num_epochs=5)

  0%|          | 90/285884 [00:32<28:49:21,  2.75it/s]


KeyboardInterrupt: 